# Selección de animal a estudiar

In [6]:
import polars as pl

In [ ]:
schema = {
    "individual_id": pl.String,
    "date": pl.String,
    "decimal_longitude": pl.Float64,
    "decimal_latitude": pl.Float64,
    "year": pl.Int32,
    "month": pl.Int8,
    "day": pl.Int8,
    "hour": pl.Int8,
}

tracking = pl.read_csv("../../data/foca_procesado.csv", schema=schema)
tracking = (
    tracking
    .with_columns(
        pl.col("date").str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S")
    )
)
tracking = tracking.sort(by = "date")
tracking = tracking.filter(pl.col("decimal_longitude") >= -90).filter(pl.col("decimal_longitude") <= 90) # Quitar outliers
tracking = tracking.filter(pl.col("year") >= 2005) # Quitar algunos años para que el dataset se pueda tratar en local

In [8]:
max_latitude = tracking["decimal_latitude"].max()
max_longitude = tracking["decimal_longitude"].max()
min_latitude = tracking["decimal_latitude"].min()
min_longitude = tracking["decimal_longitude"].min()

In [9]:
box = pl.DataFrame({"max_latitude": max_latitude, "min_latitude": min_latitude, "max_longitude": max_longitude, "min_longitude": min_longitude})
box.write_csv("../../data/world_box.csv")

In [10]:
temp = pl.DataFrame({"first": [1, 0], "day": [tracking.head(1)["date"].to_list()[0].day, tracking.tail(1)["date"].to_list()[0].day], 
                                              "month": [tracking.head(1)["date"].to_list()[0].month, tracking.tail(1)["date"].to_list()[0].month], 
                                                        "year": [tracking.head(1)["date"].to_list()[0].year, tracking.tail(1)["date"].to_list()[0].year]})
temp.write_csv("../../data/temporal_subset.csv")